# TimeSeries DataAnalytics Tutorial

## I/ Requete des données

### A/ Données SmartGrid

### B/ Données Eolienne

### C/ Données de polution

## II/ Statistiques

## III/ FFT

Notebook [FFTTuto](./fft.ipynb) illustrant l'utilisation des FFT et PSD sur des données temporelles

## IV/ Clustering

## V/ Apprentissage

